In [ ]:
import gurobipy as gp
from gurobipy import GRB

In [ ]:

dairy = ['yogurt', 'butter', 'cheese1', 'cheese2']
components = ['fat', 'dryMatter']

# Percentage composition of the products

composition, product = gp.multidict({
    ('fat','yogurt'): 0.03,
    ('fat','butter'): 0.8,
    ('fat','cheese1'): 0.35,
    ('fat','cheese2'): 0.25,
    ('dryMatter','yogurt'): 0.08,
    ('dryMatter','butter'): 0.02,
    ('dryMatter','cheese1'): 0.3,
    ('dryMatter','cheese2'): 0.4
})

# The yearly availability of components (1000 tons)

components, capacity = gp.multidict({
    ('fat'): 600,
    ('dryMatter'): 750
})

# Last year's domestic consumption and prices

dairy, consumption, price, elasticity = gp.multidict({
    ('yogurt'): [4.82, 0.297, 0.4],
    ('butter'): [0.32, 0.72, 2.7],
    ('cheese1'): [0.21, 1.05, 1.1],
    ('cheese2'): [0.07, 0.815, 0.4]
})

elasticity12 = 0.1
elasticity21 = 0.4

priceIndex = 1.939

In [ ]:
m = gp.Model('AgriculturalPricing')

# Set global parameters
m.params.nonConvex = 2

# Quantity of dairy products
q_var = m.addVars(dairy, name="qvar")

# Price of dairy products
p_var = m.addVars(dairy, name="pvar")

Restricted license - for non-production use only - expires 2026-11-23
Set parameter NonConvex to value 2


In [ ]:
# Capacity constraint:

fat_dryMatter_Capacity = m.addConstrs( (gp.quicksum(product[c,d]*q_var[d] for d in dairy)
                             <= capacity[c] for c in components  ),
                          name='fat_dryMatter_Capacity')

In [ ]:
# Price index constraint:

priceIndex = m.addConstr( (gp.quicksum(consumption[d]*p_var[d] for d in dairy) <= priceIndex )
                         , name='priceIndex')

In [ ]:
# Elasticity constraints:

elasYogurt = m.addConstr( (q_var['yogurt']-consumption['yogurt'])/consumption['yogurt']
                           == -elasticity['yogurt']*(p_var['yogurt']-price['yogurt'])/price['yogurt']
                         , name='elasYogurt')

elasButter = m.addConstr( (q_var['butter']-consumption['butter'])/consumption['butter']
                           == -elasticity['butter']*(p_var['butter']-price['butter'])/price['butter']
                         , name='elasButter')

elasCheese1 = m.addConstr( (q_var['cheese1']-consumption['cheese1'])/consumption['cheese1']
                           == -elasticity['cheese1']*(p_var['cheese1']-price['cheese1'])/price['cheese1']
                              +elasticity12*(p_var['cheese2']-price['cheese2'])/price['cheese2']
                          , name='elasCheese1')

elasCheese2 = m.addConstr( (q_var['cheese2']-consumption['cheese2'])/consumption['cheese2']
                           == -elasticity['cheese2']*(p_var['cheese2']-price['cheese2'])/price['cheese2']
                              +elasticity21*(p_var['cheese1']-price['cheese1'])/price['cheese1']
                          , name='elasCheese2')

In [ ]:
# Quadratic objective function

obj = gp.quicksum(q_var[d]*p_var[d] for d in dairy)

m.setObjective(obj, GRB.MAXIMIZE)

# Run optimization engine

m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
NonConvex  2

Optimize a model with 7 rows, 8 columns and 22 nonzeros
Model fingerprint: 0x15283257
Model has 4 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-02, 1e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+02]
Presolve removed 2 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 14 rows, 14 columns, 36 nonzeros
Presolved model has 4 bilinear constraint(s)
Variable types: 14 continuous, 0 integer (0 binary)
Found heuristic solution: objective 2.0664080

Root relaxation: objective 2.791205e+00, 10 iterations, 0.00 seconds (0

In [ ]:
import pandas as pd

# Output Report
price_demand = pd.DataFrame(columns=["Products", "Price", "Demand"])

for d in dairy:
    new_row = pd.DataFrame({
        "Products": [d],
        "Price": ['${:,.2f}'.format(round(1000*p_var[d].x))],
        "Demand": ['{:,.2f}'.format(round(1e6*q_var[d].x))]
    })
    price_demand = pd.concat([price_demand, new_row], ignore_index=True)

price_demand.index=[''] * len(price_demand)
price_demand

,Products,Price,Demand
,yogurt,$322.00,"4,659,235.00"
,butter,$422.00,"677,216.00"
,cheese1,$833.00,"265,379.00"
,cheese2,"$1,114.00","53,957.00"
